## Imports and Data Download

In [ ]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2023-8-01", end="2023-9-24", interval='15m')
dataF.iloc[-1:,:]
dataF.Open.iloc

## Defining Signal Function

> Do the last few candles have engulfing patterns?

In [ ]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open>close and previous_open<previous_close and close<previous_open and open>=previous_close):
        return 1    # Selling signal
    
    # Bullish Pattern
    elif (open<close and previous_open>previous_close and close>previous_open and open<=previous_close):
        return 2    # Buying signal
    
    # No Clear Pattern
    else:
        return 0
    
signal = []
signal.append(0)
for i in range(1, len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))

#signal_generator(data)
dataF["signal"] = signal

In [ ]:
# Print Statements

#dataF.signal.value_counts()
dataF.iloc[:, :]

## Market Connection and Trade Execution

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
from config import access_token, accountID
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)

def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', "Low"])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1, :])

    # EXECUTING ORDERS
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['Open'].iloc[-2]-dfstream['Close'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1, :])
    print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)

    # Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-10000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

    # Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=10000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

## Executing Trades Automatically

In [ ]:
# MANUAL
#trading_job()

# SCHEDULED
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,6,11,16,21,26,31,36,41,46,51,56', start_date='2023-09-26 01:00:00')
scheduler.start()